Este agarra los datos del csv con la matriz de grayscale de cada espectrograma, y se los pasa al modelo de CNN de PyTorch (definido en aux file models.py), los guarda para poder llamarlo sin re-entrenar desde otros archivos

In [1]:
import torch
import torchaudio

In [ ]:
torch.cuda.is_available()
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")